In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a chat model instance
llm = ChatOpenAI(
   temperature=0.1,
#    streaming=True,
#    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

# memory = memory.load_memory_variables({})['chat_history']
# memory

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),
])

# loadmemory = memory.load_memory_variables({})['chat_history']
# type(loadmemory)
# loadmemory

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})['chat_history']

chain = RunnablePassthrough.assign(chat_history = load_memory) | prompt | llm

# chain.invoke({
#     # "chat_history": memory.load_memory_variables({})['chat_history'], 
#     # >> 대체 : RunnablePassthrough.assign(chat_history = memory.load_memory_variables({})['chat_history'])
#     "question": "My name is wonjoo."
# })

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


AIMessage(content='Hello kk! How can I assist you today?')

In [30]:
invoke_chain("my name is wonjoo")

{'question': 'my name is wonjoo'}
content='Hello Wonjoo! How can I assist you today?'


In [31]:
invoke_chain("what is my name?")

{'question': 'what is my name?'}
content='Your name is Wonjoo.'
